In [4]:
pip install mne

     |████████████████████████████████| 6.6 MB 2.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install GoogleDriveDownloader

Note: you may need to restart the kernel to use updated packages.


In [9]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import numpy as np
import json
import mne

In [10]:
dataset_ids = {
    1: '1ym3JqI4_ZYNSBLxoF1vNxI5Hsgur_tha',
    2: '1tJ5tHbE-2jwr0gA33Gd873lRPao-e4dF',
    3: '1tXdpY-mser01POaP6Qwixny6LjsXXoXB',
    4: '1T00cliWp5yqqbbWZ7-rf2X4tOUQ-PvIQ',
    5: '1CYsBFMul9zB_vCy0XD7XVfKUl8vihDYZ',
    6: '1io6jdALBKeopELWXahPzuAi6EfYDgviW',
    7: '1YDkheRDPNDR1ujsqqC_SY6cebWHkw9Xt',
    8: '1jjoQJFDCi7O9Q-iaReAPpQnxC-HIKpQi',
}
label_id = '1mD5MXoh6tfQJFXIvdw2MQsEu6vZka6C0'
desc = '14kYNBZYdttqmSS_Vz6Bm_ztG9Uw1MC0y'

# ALTERE O ID DO DATASET DE SUA ESCOLHA AQUI ##################################
DS = 6

In [11]:
gdd.download_file_from_google_drive(file_id=dataset_ids[DS],
                                    dest_path='files/data.npy',
                                    showsize=True)
# download do arquivo de marcações
gdd.download_file_from_google_drive(file_id=label_id,
                                    dest_path='files/labels.npy', showsize=True)

# download do arquivo de descrição
gdd.download_file_from_google_drive(file_id=desc,
                                    dest_path='files/descriptor.json',
                                    showsize=True)

295.5 MiB                                                                                                      Done.
0.0 B Done.
0.0 B Done.


In [12]:
X = np.load('files/data.npy')
y = np.load('files/labels.npy')
desc_file = open('files/descriptor.json')
descriptor = json.loads(desc_file.read())
desc_file.close()
print('Estruturas => dados', X.shape, 'labels', y.shape)

Estruturas => dados (125, 257, 1205) labels (125,)


In [13]:
#Os dados estão estruturados em numpy arrays.

#O "shape" (125, 257, 1205) para os dados EEG de X, significa que existem 125 trials (tradução aceitável: ensaio, julgamento), 257 channels que representam os eletrodos e 1205 dados que representam o potencial mensurado em ponto flutuante. Este é um array com três dimensões.

#O "shape" (125,) para os dados de labels ou marcadores y, apresentam qual é o eletrodo respectivo a cada trial. Os labels estão com valores numéricos como é mostrado a seguir, o que facilita o processamento, mas não é intuitivo.

In [14]:
print('Características do voluntário:', descriptor[str(DS)])
print('\nRótulos:', descriptor['frequencies'])
print('\nTaxa de amostragem:', descriptor['sampling_rate'])

Características do voluntário: {'age': '31', 'capsize': 'Adult Medium', 'gender': 'Female', 'hairtype': 'Thick', 'handedness': 'Right'}

Rótulos: {'1': 6.66, '2': 7.5, '3': 8.57, '4': 10.0, '5': 12.0}

Taxa de amostragem: 250


In [23]:
X = X[:,:256,:]
ch_names = X.shape[1]
sfreq = X.shape[-1]
ch_types = 'eeg'

info = mne.create_info(ch_names=ch_names, sfreq=250.0, ch_types=ch_types)

In [24]:
events = [[i, 0, e] for i, e in enumerate(y)]
obj = mne.EpochsArray(X, info, events=np.array(events))

125 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


In [25]:
obj

<EpochsArray  |   125 events (all good), 0 - 4.816 sec, baseline off, ~294.7 MB, data loaded,
 '1': 25
 '2': 25
 '3': 30
 '4': 25
 '5': 20>

In [ ]:

import matplotlib

%matplotlib inline
# aumentando a escala das figuras no matplotlib (padrão 6. x 4.)
matplotlib.rcParams['figure.figsize'] = [12, 8]
import mne

# objeto com o nome dos canais pelo modelo da touca utilizada
montage = mne.channels.make_standard_montage('EGI_256')
# criando o objeto `info` (o restante dos valores já temos)
info = mne.create_info(montage.ch_names,
                       sfreq=descriptor['sampling_rate'],
                       ch_types='eeg')
info.set_montage(montage)
info


# plotando o padrão EGI 256
print('EGI 256')
montage.plot()
# plotando o padrão 10-20
print('10-20 SYSTEM')
# neste padrão a figura estava ficando maior, então diminuí
matplotlib.rcParams['figure.figsize'] = [10, 7]
mne.channels.make_standard_montage('standard_1020').plot()
print()

X = X[:,:256,:]
# objeto event é uma matriz tridimensional conforme explicado em aula
events = np.array([[i, 0, e] for i, e in enumerate(y)])
# instanciando objeto EpochArray
epoch = mne.EpochsArray(X, info, events=events)
epoch

epoch_ex = epoch.copy().pick_channels(['E116', 'E126', 'E150'])
epoch_ex.filter(l_freq=4.0, h_freq=14.0)
epoch_ex

print(epoch.get_data().shape)
print(epoch_ex.get_data().shape)

matplotlib.rcParams['figure.figsize'] = [6., 4.]

# escolhendo (filtrando) somente o evento '1'
e = '1'
# selecionando alguns trials aleatórios (3, 9 e 11)
for i in (3, 9, 11):
    epoch_ex[e][-i].plot_psd()
# mostrando todos os trials do evento '1'
matplotlib.rcParams['figure.figsize'] = [12., 8.]
epoch_ex[e].plot_psd()
print()